In [ ]:
import meshio

mesh = meshio.read("meshes/Cuboid1.msh")
"triangle" in mesh.cells[0].type

In [ ]:
from solver import solve
import numpy as np

# position = np.array([-0.00181, -0.000408, 0.00159])
position = np.array([0.015, 0.0, 0.04])
force = np.array([-1, 0, 0])
loads = [(position, force)]

# radius = 0.00025
radius = 0.0025
msh = "meshes/Cuboid1.msh"
domain, vm, uh = solve(loads, radius, msh, debug=True)

In [ ]:
from dolfinx import plot
import pyvista

topology, cell_types, geometry = plot.vtk_mesh(domain, domain.topology.dim)
warped_geometry = geometry + uh.x.array.reshape(-1, 3)
warped_grid = pyvista.UnstructuredGrid(topology, cell_types, warped_geometry)
warped_grid.point_data["VonMises"] = vm.x.array

# Slice at the bottom to get a flat cross-section
z_min = warped_geometry[:, 2].min()
z_slice = z_min + (warped_geometry[:, 2].max() - z_min) * 0.001  # Slice near bottom
bottom_slice = warped_grid.slice(normal=[0, 0, 1], origin=[0, 0, z_slice])

plotter = pyvista.Plotter()
plotter.add_mesh(warped_grid, opacity=1, scalar_bar_args={"vertical": True})
plotter.show_axes()
plotter.show()

In [ ]:
%load_ext autoreload
%autoreload 2

from simulator import Simulator
import numpy as np

# position = np.array([-0.00181, -0.000408, 0.00159])
position = np.array([-0.00462995  0.02232599  0.00878814])
force = np.array([0.916213, 0.09119986, 0.39017478]) * 1
loads = [(position, force)]

# radius = 0.0005
radius = 0.005
msh = "meshes/HexNuts/msh/00000006_cg2.msh"
# msh = "meshes/00000006.msh"
simulator = Simulator(msh, contact_radius=radius)
# simulator.plot_patch(loads)
uh = simulator.run(loads)
# simulator.plot_displacement(uh)
vm = simulator.compute_vm1(uh)
simulator.plot_vm(vm)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Info    : Reading 'meshes/HexNuts/msh/00000006_cg2.msh'...
Info    : 69 entities
Info    : 19143 nodes
Info    : 15281 elements
Info    : Done reading 'meshes/HexNuts/msh/00000006_cg2.msh'

Applying force [0.916213   0.09119986 0.39017478] on tag 2 with area 5.210197e-05
  Normalized force: [17584.99871499  1750.41111718  7488.67676503]


Widget(value='<iframe src="http://localhost:60270/index.html?ui=P_0x3936ca390_5&reconnect=auto" class="pyvista…

In [6]:
simulator.plot_vm_bottom(vm)

Widget(value='<iframe src="http://localhost:59653/index.html?ui=P_0x332a6b290_5&reconnect=auto" class="pyvista…

In [4]:
from solver import solve

domain, uh, vm = solve(loads, radius, msh, debug=True)


Domain bounds: [-2.16498477e-02 -2.49990908e-02  9.09179957e-07] to [0.02164985 0.02499909 0.01443414] (m)
bottom_facets: 770 bottom_dofs: 1654


Widget(value='<iframe src="http://localhost:60270/index.html?ui=P_0x3862e2c00_1&reconnect=auto" class="pyvista…